<a href="https://colab.research.google.com/github/rajeevfromkrec/computer-vision/blob/master/yolo_v3_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget https://pjreddie.com/media/files/yolov3.weights



--2020-07-12 12:04:52--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  1.43MB/s    in 4m 38s  

2020-07-12 12:09:31 (872 KB/s) - ‘yolov3.weights’ saved [248007048/248007048]

--2020-07-12 12:09:32--  https://github.com/pjreddie/darknet/blob/master/data/coco.names
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘coco.names.1’

coco.names.1            [ <=>                ]  93.79K  --.-KB/s    in 0.08s   

2020-07-12 12:09:33 (1.22 MB/s) - ‘coco.names.1’ saved [96039]



In [11]:
!git clone https://github.com/pjreddie/darknet.git

fatal: destination path 'darknet' already exists and is not an empty directory.


In [16]:
import os
!cp ./darknet/cfg/yolov3.cfg ./yolov3.cfg
!cp ./darknet/data/coco.names ./coco.names


In [ ]:
!cat coco.names

In [ ]:
import tensorflow as tf
from tensorflow.keras import model
from tensorflow.keras.layers import BatchNormalization, Conv2D, Input,ZeroPadding2D,LeakyReLU,UpSampling2D


In [ ]:
def parse_cfg(cfgfile):
  with open(cfgfile,'r' ) as file:
    lines = [line.rstrip('\n') for line in file if line != '\n' and line[0] != '#']
  holder= {}
  blocks= []
  for line in lines:
    if line[0]=='[':
      line='type=' + line[1:-1].rstrip()
      if len(holder)!=0:
        blocks.append(holder)
        holder={}
    key,value= line.split('=')
    holder[key.rstrip()] = value.lstrip() 
  blocks.append(holder) 
  return blocks                 

fblocks= parse_cfg('yolov3.cfg') 

print(fblocks)

In [ ]:
def YOLOV3net(cfgfile,model_size,num_classes):
  blocks=parse_cfg(cfgfile)
  output= {}
  output_filters=[]
  filters=[]
  scale=0
  inputs=input_image=Input(shape=model_size)
  inputs=inputs/255